In [2]:
import numpy as np
import pandas as pd

In [30]:
data = pd.read_excel('C:\\Users\\Asus\\Documents\\SEM VI\\CITI\\Project\\data.xlsx', index = False)
data[:5]

,Unique Id,Client Name,Account ID,Legal Entity,Currency,Payment Type,Paid Amount,Payment Date,Payment Status,Pending Amount,Comments
0,1,J P Morgan,101008,CitiBank New York Branch,GBP,Receive,45678,2020-03-11,Fully paid,0,Transaction complete
1,2,J P Morgan,105049,CitiBank New York Branch,USD,Receive,765327,2020-03-12,Partially Paid,1008,Waiting for pending amount
2,3,DEUTSCHE BANK,105050,CitiBank HongKong,CAD,Receive,637229,2020-03-13,Partially Paid,34566,Waiting for pending amount
3,4,DEUTSCHE BANK,105058,CitiBank New York Branch,USD,Receive,22345566,2020-03-14,Partially Paid,1000000,Waiting for pending amount
4,5,DEUTSCHE BANK,105059,CitiBank New York Branch,EUR,Receive,553322,2020-03-15,Rejected,553322,Payment Failed


In [31]:
data['Account ID'].describe(), data.shape

(count        20.000000
 mean     104901.450000
 std         917.150247
 min      101008.000000
 25%      105060.500000
 50%      105117.500000
 75%      105128.250000
 max      105170.000000
 Name: Account ID, dtype: float64, (20, 11))

In [32]:
import os

location = 'C:\\Users\\Asus\\Documents\\SEM VI\\CITI\\Project\\Data'
files_list = os.listdir(location)
DATASET = data.drop(columns=['Unique Id'], axis = 1)
# DATASET.rename(columns={'ClientÂ  Name' : 'ClientName'}, inplace = True)

print(DATASET.columns)

for file in files_list:
    name = os.path.join(location, file)
    df = pd.read_excel(name, index = False)
    DATASET = pd.concat([DATASET, df], axis = 0, ignore_index=True)


Index(['Client Name', 'Account ID', 'Legal Entity', 'Currency', 'Payment Type',
       'Paid Amount', 'Payment Date', 'Payment Status', 'Pending Amount',
       'Comments'],
      dtype='object')


In [33]:
print(DATASET['Payment Status'].unique(), DATASET['Comments'].unique())

['Fully paid' 'Partially Paid' 'Rejected' 'Processing'] ['Transaction complete' 'Waiting for pending amount' 'Payment Failed'
 'Payment is in progress']


## Preprocessing

#### 1) Removing redundant ACCOUNT IDs.
#### 2) Aligning the Payment Status with Paid Amount and Comments.

    I WHEN PAYMENT STATUS = Fully paid, COMMENTS = 'Transaction Complete' & PENDING AMOUNT = 0
    II WHEN PAYMENT STATUS = Partially Paid, COMMENTS = 'Waiting for pending amount' 
    III WHEN PAYMENT STATUS = Rejected, COMMENTS = 'Payment Failed' & PENDING AMOUNT = PAID AMOUNT
    IV WHEN PAYMENT STATUS = Processing, COMMENTS = 'TPayment is in progress' & PENDING AMOUNT = PAID AMOUNT
    
#### 3) Resetting the index

In [34]:
DATASET.drop_duplicates(['Account ID'], inplace=True)

# FIRST CASE

DATASET.loc[DATASET["Payment Status"] == 'Fully paid', 'Pending Amount'] = 0
DATASET.loc[DATASET["Payment Status"] == 'Fully paid', 'Comments'] = 'Transaction complete'

# SECOND CASE

DATASET.loc[DATASET["Payment Status"] == 'Partially Paid', 'Comments'] = 'Waiting for pending amount'

# THIRD CASE

DATASET.loc[DATASET["Payment Status"] == 'Rejected', 'Pending Amount'] = DATASET.loc[DATASET["Payment Status"] == 'Rejected', 'Paid Amount']
DATASET.loc[DATASET["Payment Status"] == 'Rejected', 'Comments'] = 'Payment Failed'

# FOURTH CASE

DATASET.loc[DATASET["Payment Status"] == 'Processing', 'Pending Amount'] = DATASET.loc[DATASET["Payment Status"] == 'Processing', 'Paid Amount']
DATASET.loc[DATASET["Payment Status"] == 'Processing', 'Comments'] = 'Payment is in progress'

DATASET.reset_index(drop=True, inplace=True)

In [35]:
DATASET

,Client Name,Account ID,Legal Entity,Currency,Payment Type,Paid Amount,Payment Date,Payment Status,Pending Amount,Comments
0,J P Morgan,101008,CitiBank New York Branch,GBP,Receive,45678,2020-03-11 00:00:00,Fully paid,0,Transaction complete
1,J P Morgan,105049,CitiBank New York Branch,USD,Receive,765327,2020-03-12 00:00:00,Partially Paid,1008,Waiting for pending amount
2,DEUTSCHE BANK,105050,CitiBank HongKong,CAD,Receive,637229,2020-03-13 00:00:00,Partially Paid,34566,Waiting for pending amount
3,DEUTSCHE BANK,105058,CitiBank New York Branch,USD,Receive,22345566,2020-03-14 00:00:00,Partially Paid,1000000,Waiting for pending amount
4,DEUTSCHE BANK,105059,CitiBank New York Branch,EUR,Receive,553322,2020-03-15 00:00:00,Rejected,553322,Payment Failed
5,BNY Mellon,105061,CitiBank London,USD,Receive,56789,2020-03-16 00:00:00,Rejected,56789,Payment Failed
6,BNY Mellon,105071,CitiBank London,GBP,Receive,98735627,2020-03-07 00:00:00,Rejected,98735627,Payment Failed
7,BNY Mellon,105101,CitiBank London,EUR,Receive,6578,2020-03-08 00:00:00,Fully paid,0,Transaction complete
8,BNY Mellon,105102,CitiBank New York Branch,USD,Receive,352627,2020-03-09 00:00:00,Partially Paid,23456,Waiting for pending amount
9,J P Morgan,105115,CitiBank Singapore,EUR,Receive,532828,2020-03-10 00:00:00,Fully paid,0,Transaction complete


### Processing the Date Column

###### Converting the DATETIME datatype into string for usage later.

In [36]:
from datetime import datetime

DATASET['Payment Date'][20:] = DATASET['Payment Date'][20:].apply(lambda data : datetime.strptime(str(data), "%Y/%m/%d"))
DATASET['Payment Date'] = DATASET['Payment Date'].apply(lambda data : str(data)[:11])

type(DATASET['Payment Date'][11])

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


str

In [37]:
DATASET.to_csv('C:\\Users\\Asus\\Documents\\SEM VI\\CITI\\Project\\dataset.csv')

In [20]:
questions = pd.read_pickle('C:\\Users\\Asus\\Documents\\SEM VI\\CITI\\Project\\questions.pkl')